In [1]:
import cobra
import pickle
import numpy as np

In [12]:
def test_growth(model):
    model.objective = 'Biomass_v1'
    model.objective_direction = 'max'
    s = model.optimize()
    r = s.objective_value if s.status == 'optimal' else -1
    print('Results:')
    print('  Status        :',s.status)
    print('  growth_rate   :',r)
    print('  glucose uptake:',s.fluxes['Exchange_Glucopyranose'])
    print('  PHA           :',s.fluxes['PHA_secretion'])
    print('  NGAM lb:',model.reactions.NGAM.lower_bound)
    print('  PHA lb:',model.reactions.PHA_secretion.lower_bound)
    
    print()
            

### For Batch

In [13]:
from curate_kcat_light_goslim_asc_step2 import *
posterior = pickle.load(open('../Results/smc_abc_light_go_and_one_asc_step_2.pkl','rb'))

In [15]:
# prepare models with kcats from step2, but only constrain total protein content
tmp_model_file = '../Results/template_ecModel_goslim_pools.pkl'
tmp_model = pickle.load(open(tmp_model_file,'rb'))

dffrac = pd.read_csv('../Results/protein_abundance_go_slim_level_uniq_asc.csv',index_col=0)
dftot = pd.read_csv('../proteomics/total_protein_abandance_mean.csv',index_col=0)
dfpheno = pd.read_csv('../proteomics/phynotype.csv',index_col=0,comment='#')

_,pools = buildPoolDict(dffrac,'NACL60',dftot.loc['NACL60','Ptot'],sigma=0.5)

models = []
for kcat_dct in posterior.population: # there are 100 kcat sets in the population
    model = tmp_model.copy()
    updateKcats(model,kcat_dct)
    updateProteinConstraints(model,pools)
    
    test_growth(model)
    models.append(model)


Results:
  Status        : optimal
  growth_rate   : 0.15271857512943499
  glucose uptake: 0.522515443016676
  PHA           : 0.0
  NGAM lb: 6.86
  PHA lb: 0.0

Results:
  Status        : optimal
  growth_rate   : 0.14839696094570626
  glucose uptake: 0.5141488778437546
  PHA           : 14.131294810581624
  NGAM lb: 6.86
  PHA lb: 0.0

Results:
  Status        : optimal
  growth_rate   : 0.13171648240062506
  glucose uptake: 0.3484343369798035
  PHA           : 0.0
  NGAM lb: 6.86
  PHA lb: 0.0

Results:
  Status        : optimal
  growth_rate   : 0.14663255850952778
  glucose uptake: 0.266910197879376
  PHA           : 0.0
  NGAM lb: 6.86
  PHA lb: 0.0

Results:
  Status        : optimal
  growth_rate   : 0.15049913558454545
  glucose uptake: 0.289001664505746
  PHA           : 0.0
  NGAM lb: 6.86
  PHA lb: 0.0

Results:
  Status        : optimal
  growth_rate   : 0.150536358079175
  glucose uptake: 0.23689675015175155
  PHA           : 0.0
  NGAM lb: 6.86
  PHA lb: 0.0

Results:
  

KeyboardInterrupt: 

### For fed-batch

In [19]:
# prepare models with kcats from step2, but only constrain total protein content
tmp_model_file = '../Results/template_ecModel_goslim_pools.pkl'
tmp_model = pickle.load(open(tmp_model_file,'rb'))

dffrac = pd.read_csv('../Results/protein_abundance_go_slim_level_uniq_asc.csv',index_col=0)
dftot = pd.read_csv('../proteomics/total_protein_abandance_mean.csv',index_col=0)
dfpheno = pd.read_csv('../proteomics/phynotype.csv',index_col=0,comment='#')

Ptot = np.mean(dftot.loc[['Fermentation-9h','Fermentation-19h','Fermentation-30h'],:].values)
_,pools = buildPoolDict(dffrac,'Fermentation-19h',Ptot,sigma=0.5)

models = []
for kcat_dct in posterior.population: # there are 100 kcat sets in the population
    model = tmp_model.copy()
    updateKcats(model,kcat_dct)
    updateProteinConstraints(model,pools)
    
    test_growth(model)
    models.append(model)


Results:
  Status        : optimal
  growth_rate   : 0.1247495349644616
  glucose uptake: 0.0932124704824853
  PHA           : 0.0
  NGAM lb: 6.86
  PHA lb: 0.0

Results:
  Status        : optimal
  growth_rate   : 0.11834784222961711
  glucose uptake: 0.08847313861844991
  PHA           : 13.304448860635302
  NGAM lb: 6.86
  PHA lb: 0.0

Results:
  Status        : optimal
  growth_rate   : 0.1102951236873927
  glucose uptake: 0.08245391385564273
  PHA           : 0.0
  NGAM lb: 6.86
  PHA lb: 0.0

Results:
  Status        : optimal
  growth_rate   : 0.10422178677567565
  glucose uptake: 0.08283752425654076
  PHA           : 7.824547247054047
  NGAM lb: 6.86
  PHA lb: 0.0

Results:
  Status        : optimal
  growth_rate   : 0.12738656659005365
  glucose uptake: 0.09523021990119351
  PHA           : 0.0
  NGAM lb: 6.86
  PHA lb: 0.0



KeyboardInterrupt: 